In [1]:
import pickle
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
with open('data_theta_profile.pkl', 'rb') as f:
    profile =  pickle.load(f)
with open('data_inversion_label.pkl', 'rb') as f:
    label =  pickle.load(f)


In [5]:
print(profile[20,:])

[380.28833108 362.73410185 354.15751345 348.28570231 346.22247036
 345.95875793 345.59312562 342.57569336 338.6175886  334.44733119
 330.41748071 325.9423819  321.97828963 317.59841766 314.07576326
 311.31011697 308.79610548 307.56839847 306.44848267 305.20941576
 303.92724913 302.58956468 301.34746163 300.24697176 299.01416612
 298.14466883 297.52420583]


In [8]:
x_train, x_test, y_train, y_test = train_test_split(profile, label, test_size=0.25, random_state=69)

In [12]:
print(x_train.shape)
print(x_train[50,:])

(7062, 27)
[372.20140846 360.61262021 357.98419686 354.20243778 348.55118115
 347.10844388 346.14459215 343.08171491 338.46196948 332.64297328
 329.39374079 324.55574733 320.44083302 317.26264349 313.83384178
 309.92373366 306.1850784  303.45498984 300.99869309 298.82663196
 296.48006036 295.17412018 294.12495291 292.64614405 292.56025555
 292.54028215 292.59272939]


In [13]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [14]:
print(x_train[50,:])

[-0.98414262 -0.60390941  0.53276102  0.69556051 -0.32079324  0.06434833
  0.48403155  0.78072617  0.63502156  0.19630399  0.46908803  0.23758697
  0.17870108  0.36344551  0.3639555   0.17708065  0.05222715 -0.2015483
 -0.34701148 -0.40505704 -0.52026552 -0.49208257 -0.47755523 -0.61516153
 -0.50649939 -0.44193359 -0.41503815]


In [16]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [19]:
## train data
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(x_train), 
                       torch.FloatTensor(y_train))
## test data    
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = TestData(torch.FloatTensor(x_test))

In [20]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64)

In [30]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()        # Number of input features is 12.
        self.layer_1 = nn.Linear(27, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
###################### OUTPUT ######################cuda:0

cpu


In [32]:
model = BinaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BinaryClassification(
  (layer_1): Linear(in_features=27, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [33]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [34]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.58209 | Acc: 68.495
Epoch 002: | Loss: 0.49380 | Acc: 75.351
Epoch 003: | Loss: 0.46759 | Acc: 76.919
Epoch 004: | Loss: 0.46196 | Acc: 76.991
Epoch 005: | Loss: 0.45780 | Acc: 77.477
Epoch 006: | Loss: 0.44870 | Acc: 77.721
Epoch 007: | Loss: 0.44877 | Acc: 77.631
Epoch 008: | Loss: 0.43927 | Acc: 78.297
Epoch 009: | Loss: 0.44116 | Acc: 78.090
Epoch 010: | Loss: 0.43145 | Acc: 78.937
Epoch 011: | Loss: 0.43491 | Acc: 79.018
Epoch 012: | Loss: 0.42818 | Acc: 79.450
Epoch 013: | Loss: 0.43112 | Acc: 79.270
Epoch 014: | Loss: 0.42402 | Acc: 79.459
Epoch 015: | Loss: 0.42668 | Acc: 79.495
Epoch 016: | Loss: 0.42136 | Acc: 79.568
Epoch 017: | Loss: 0.41454 | Acc: 80.360
Epoch 018: | Loss: 0.41418 | Acc: 80.099
Epoch 019: | Loss: 0.41377 | Acc: 80.369
Epoch 020: | Loss: 0.40975 | Acc: 80.045
Epoch 021: | Loss: 0.41310 | Acc: 80.613
Epoch 022: | Loss: 0.40519 | Acc: 80.342
Epoch 023: | Loss: 0.40347 | Acc: 80.811
Epoch 024: | Loss: 0.40281 | Acc: 80.532
Epoch 025: | Los

In [37]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [44]:
print(len(y_pred_list))

2355


In [38]:
confusion_matrix(y_test, y_pred_list)

array([[ 346,  325],
       [ 227, 1457]], dtype=int64)

In [39]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.60      0.52      0.56       671
           1       0.82      0.87      0.84      1684

    accuracy                           0.77      2355
   macro avg       0.71      0.69      0.70      2355
weighted avg       0.76      0.77      0.76      2355

